In [1]:
# Time series analysis of daily stock prices
# Decomposition into seasonal, trend and residual components
# Autocorrelation, ARIMA and SARIMAX model for forecasting

import numpy as np
import pandas as pd
from scipy import signal
import statsmodels.tsa as tsa #.stattools
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import acf, pacf
import matplotlib.pyplot as plt
%matplotlib inline 
plt.style.use('seaborn-darkgrid')
# print(plt.style.available)
import colorlover as cl
from IPython.display import HTML
import plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode()

In [2]:
import pandas_datareader as pdr
from datetime import datetime
ticker = 'ADBE'
# get past two years data
data = pdr.get_data_yahoo(symbols=ticker, start=datetime(2015, 1, 1), end=datetime(2017, 1, 24))
dataAC = (data['Adj Close'])

In [3]:
daily = data.set_index(data.index).groupby(pd.TimeGrouper(freq='D'))['Close'].mean()
new = daily.fillna(method='ffill')

In [4]:
Y = new

In [5]:
# Plot data to have a look
# Create traces
trace0a = go.Scatter(
    x = Y.index,
    y = Y,
    mode = 'line',
    name = 'daily',
    marker = dict(
        size = 7,
        color = '#50C5B7',
        line = dict(
            width = 2,
            color = '#50C5B7'
        ) 
        )
)  
layout1 = go.Layout(
         #title = 'Timeseries: '+str(startday)+' - '+str(endday),
         xaxis = dict(
             #range = ['2000', '2017'],
             rangeslider = dict(thickness = 0.05),
             showline = True,
             showgrid = False
         ),
         yaxis = dict(
             #range = [0, 10000],
             showline = True,
             showgrid = False)
         )

data = [trace0a]
fig = dict(data=data, layout=layout1)
iplot(fig)

# Test: Stationarity

In [6]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    #Determing rolling statistics
    rolmean = timeseries.rolling(window=12,center=False).mean()
    rolstd = timeseries.rolling(window=12,center=False).std()

    #Plot rolling statistics:
    trace01 = {'x': timeseries.index, 'y': timeseries, 'type': 'scatter', 'name': 'Original'}
    trace02 = {'x': rolmean.index, 'y': rolmean, 'type': 'scatter', 'name': 'Rolling mean'}
    trace03 = {'x': rolstd.index, 'y': rolstd, 'type': 'scatter', 'name': 'Rolling Std' }
    layout1 = go.Layout(
         title = timeseries.name + 'Rolling Mean & Standard Deviation',
         #marker=dict(colorscale="Viridis"),
         xaxis = dict(
            rangeslider = dict(thickness = 0.05),
            showline = True,
         ),
         yaxis = dict(
             #range = [0, 10000],
             showline = True,
             showgrid = False,
        )
        )
    data = [trace01,trace02,trace03]
    fig = dict(data=data, layout=layout1)
    iplot(fig)

    #Perform Dickey-Fuller test:
    print 'Results of Dickey-Fuller Test:'
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print dfoutput 

### Rolling average

In [7]:
Y.name = 'Timeseries: '
test_stationarity(Y)

Results of Dickey-Fuller Test:
Test Statistic                  -0.740994
p-value                          0.835844
#Lags Used                       7.000000
Number of Observations Used    746.000000
Critical Value (5%)             -2.865422
Critical Value (1%)             -3.439146
Critical Value (10%)            -2.568837
dtype: float64


### Log stationarity

In [8]:
Y_log = np.log(Y)
Y_log.name = 'TS log: '
Y_log = Y_log.replace([np.inf, -np.inf], 0)
Y_log.dropna(inplace=True)
moving_avg = Y_log.rolling(window=12,center=False).mean()
test_stationarity(Y_log)

Results of Dickey-Fuller Test:
Test Statistic                  -1.016667
p-value                          0.747137
#Lags Used                       7.000000
Number of Observations Used    746.000000
Critical Value (5%)             -2.865422
Critical Value (1%)             -3.439146
Critical Value (10%)            -2.568837
dtype: float64


### Log difference

In [9]:
Y_log_moving_avg_diff = Y_log - moving_avg
Y_log_moving_avg_diff.dropna(inplace=True)
Y_log_moving_avg_diff.name = 'Log moving avg. diff. - '
test_stationarity(Y_log_moving_avg_diff) #doesn't pass

Results of Dickey-Fuller Test:
Test Statistic                -9.428958e+00
p-value                        5.252952e-16
#Lags Used                     5.000000e+00
Number of Observations Used    7.370000e+02
Critical Value (5%)           -2.865470e+00
Critical Value (1%)           -3.439254e+00
Critical Value (10%)          -2.568863e+00
dtype: float64


### Weighted average

In [10]:
expweighted_avg = Y_log.ewm(halflife=12,ignore_na=False,min_periods=0,adjust=True).mean()
Y_log_ewma_diff = Y_log - expweighted_avg
Y_log_diff = Y_log - Y_log.shift()
Y_log_diff.dropna(inplace=True)
Y_log_ewma_diff.name = "Weighted avg diff - "
test_stationarity(Y_log_ewma_diff) 

Results of Dickey-Fuller Test:
Test Statistic                -6.178635e+00
p-value                        6.536862e-08
#Lags Used                     3.000000e+00
Number of Observations Used    7.500000e+02
Critical Value (5%)           -2.865401e+00
Critical Value (1%)           -3.439099e+00
Critical Value (10%)          -2.568826e+00
dtype: float64


In [11]:
Y_log_diff = Y_log - Y_log.shift()
Y_log_diff.index = Y.index
Y_log_diff.dropna(inplace=True)
Y_log_diff.name = "Log diff - "
test_stationarity(Y_log_diff)

Results of Dickey-Fuller Test:
Test Statistic                -1.217574e+01
p-value                        1.387635e-22
#Lags Used                     6.000000e+00
Number of Observations Used    7.460000e+02
Critical Value (5%)           -2.865422e+00
Critical Value (1%)           -3.439146e+00
Critical Value (10%)          -2.568837e+00
dtype: float64


## Timeseries decomposition

In [12]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(Y)
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

In [13]:
# Plot decomposed timeseries
fig = tls.make_subplots(rows=4, cols=1, shared_xaxes=True)
fig.append_trace({'x': Y_log.index, 'y': Y_log, 'type': 'scatter', 'name': 'Raw'}, 1, 1)
fig.append_trace({'x': Y_log.index, 'y': seasonal, 'type': 'scatter', 'name': 'Seasonal'}, 2, 1)
fig.append_trace({'x': Y_log.index, 'y': residual, 'type': 'scatter', 'name': 'Residual'}, 3, 1)
fig.append_trace({'x': Y_log.index, 'y': trend, 'type': 'scatter', 'name': 'Trend'}, 4, 1)

layout1 = go.Layout(
         title = 'UFO Reports - timeseries decomposition',
         xaxis = dict(
             rangeslider = dict(thickness = 0.05),
             showline = True,
             showgrid = False
         ),
         yaxis = dict(
             #range = [0, 10000],
             showline = True,
             showgrid = False)
         )

fig['layout'].update(title='UFO Reports - timeseries decomposition')
iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]



### Residual timeseries

In [14]:
Y_log_decompose = residual
Y_log_decompose.name = "Residual: "
Y_log_decompose.dropna(inplace=True)
test_stationarity(Y_log_decompose)

Results of Dickey-Fuller Test:
Test Statistic                -1.013467e+01
p-value                        8.733087e-18
#Lags Used                     2.000000e+01
Number of Observations Used    7.270000e+02
Critical Value (5%)           -2.865524e+00
Critical Value (1%)           -3.439377e+00
Critical Value (10%)          -2.568891e+00
dtype: float64


# Test: Autocorrelation

In [15]:
# # Autocorrelation 
lag_acf = acf(Y, nlags=20)
lag_pacf = pacf(Y, nlags=20, method='ols')
trace1 = go.Scatter(
    x = range(0,len(lag_acf)),
    y = lag_acf,
    mode = 'lines',
    name = 'Autocorrelation',
    marker = dict(
        size = 4,
        color = '#6184D8',
        line = dict(
            width = 2,
            color = '#6184D8',
        ) 
        )
)
trace2 = go.Scatter(
    x = range(0,len(lag_acf)),
    y = lag_pacf,
    mode = 'lines',
    name = 'Partial autocorrelation',
    marker = dict(
        size = 4,
        color = '#50C5B7',
        line = dict(
            width = 2,
            color = '#50C5B7',
        ) 
        )
)
## Lines
trace3 = go.Scatter(
    x = range(0,len(lag_acf)),
    y = np.ones(len(lag_acf)) * np.array(-1.96/np.sqrt(len(Y_log_diff))),
    mode = 'lines',
    name = 'Lower',
    showlegend=False,
    marker = dict(
        size = 4,
        color = '#CCCCCC',
        opacity= 0.3,
        line = dict(
            width = 2,
            color = '#CCCCCC',
            #dash = 'dashdot'
        ) 
        )
)
trace4 = go.Scatter(
    x = range(0,len(lag_acf)),
    y = np.ones(len(lag_acf)) * np.array(1.96/np.sqrt(len(Y_log_diff))),
    mode = 'lines',
    name = 'Upper',
    showlegend=False,
    marker = dict(
        size = 4,
        color = '#CCCCCC',
        opacity= 0.3,
        line = dict(
            width = 2,
            color = '#CCCCCC',
            #dash = 'dashdot'
        ) 
        )
)

layout = go.Layout(
    title='Autocorrelation function',
    xaxis=dict(
        title='Lag',
        titlefont=dict(
            family='Arial',
            size=16,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Corr',
        titlefont=dict(
            family='Arial',
            size=16,
            color='#7f7f7f'
        )
    )
) 
data = [trace3,trace4,trace1,trace2]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

## ARIMA Model

In [16]:
# Either seed order from pacf or run grid search below to find best p,q,d params

# from pandas import datetime
# from matplotlib import pyplot
# from statsmodels.tsa.arima_model import ARIMA
# from sklearn.metrics import mean_squared_error
 
# def parser(x):
#     return datetime.strptime('190'+x, '%Y-%m')

# X = Y_log
# size = int(len(X) * 0.90)
# train, test = X[0:size], X[size:len(Y)]

# history = [x for x in train]
# predictions = list()
# forec = list()
# for t in range(len(test)):
#     model = ARIMA(history, order=(2,1,0))
#     model_fit = model.fit(disp=0)
#     output = model_fit.forecast()
#     fores = model_fit.predict()
#     yhat = output[0]
#     xhat = fores[0]
#     predictions.append(float(yhat))
#     forec.append(float(xhat))
#     obs = test[t]
#     history.append(obs)
#     #print('predicted=%f, expected=%f' % (yhat, obs))
# error = mean_squared_error(test, predictions)
# print('Test MSE: %.3f' % error)

### Fit model

In [17]:
m = ARIMA(Y, order=(4,1,0))
mf = m.fit(disp=-1)
mfore = mf.forecast()
print mf.summary()

                             ARIMA Model Results                              
Dep. Variable:         D.Timeseries:    No. Observations:                  753
Model:                 ARIMA(4, 1, 0)   Log Likelihood               -1136.932
Method:                       css-mle   S.D. of innovations              1.095
Date:                Sat, 28 Jan 2017   AIC                           2285.864
Time:                        12:32:20   BIC                           2313.608
Sample:                    01-03-2015   HQIC                          2296.552
                         - 01-24-2017                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0549      0.039      1.402      0.161      -0.022       0.132
ar.L1.D.Timeseries:      0.0144      0.037      0.394      0.693      -0.057       0.086
ar.L2.D.Time

# Forecast 2014-2016

In [18]:
# predict the missing past 3 years data
pred_steps = 90
predday = "2020-01-01"
idxN = pd.date_range(Y.index[0], predday, freq="D")
idxO = pd.date_range(Y.index[0], Y.index[(len(Y)-1)], freq="D")
#futurem = len(idxN)-len(idxO)+1

mp_res = mf.predict(len(Y_log),len(Y_log)+pred_steps,dynamic = True)
mf_res = mf.forecast(steps=pred_steps+1)
mf_err = pd.DataFrame(mf_res[2])

In [19]:
# Revert from log
# Y_orig = np.exp(Y_log)
# mf_err1_orig = np.exp(mf_err[0])
# mf_err2_orig = np.exp(mf_err[1])
# mf_res_orig = np.exp(mf_res[0])


Y_orig = (Y)
mf_err1_orig = (mf_err[0])
mf_err2_orig = (mf_err[1])
mf_res_orig = (mf_res[0])

In [20]:
# Plot ARIMA Prediction
# or
# plot ARIMA train/test result
trace1 = go.Scatter(
    x = idxN[0:len(Y_log)],
    y = Y,
    mode = 'lines',
    name = 'Training data',
    marker = dict(
        size = 4,
        color = '#6184D8',
        line = dict(
            width = 2,
            color = '#6184D8',
        ) 
        )
)
trace2 = go.Scatter(
    x = idxN[len(Y_log):len(Y_log)+pred_steps+1],
    y = mf_res_orig,
    mode = 'lines',
    name = 'Forecast',
    marker = dict(
        size = 4,
        color = '#50C5B7',
        line = dict(
            width = 2,
            color = '#50C5B7'
        ) 
        )
)
trace3 = go.Scatter(
    x = idxN[len(Y_log):len(Y_log)+pred_steps+1],
    y = mf_err1_orig,
    mode = 'lines',
    name = 'CI',
    marker = dict(
        size = 4,
        color = '#CCCCCC',
        line = dict(
            width = 2,
            color = '#CCCCCC'
        ) 
        )
)
trace4 = go.Scatter(
    x = idxN[len(Y_log):len(Y_log)+pred_steps+1],
    y = mf_err2_orig,
    mode = 'lines',
    name = 'Conf. Interval',
    showlegend=False,
    marker = dict(
        size = 4,
        color = '#CCCCCC',
        line = dict(
            width = 2,
            color = '#CCCCCC'
        ) 
        )
)
layout = go.Layout(
    title=str(ticker)+': ARIMA Model forecast',
    xaxis=dict(
        title='Date',
        titlefont=dict(
        )
    ),
    yaxis=dict(
        title='',
        #range = (0,100000), #max(Y)+100),
        titlefont=dict(
        )
    )
) 

data = [trace1,trace2,trace3,trace4]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [21]:
# Print future predictions
print_df = pd.DataFrame(mf_res_orig, index=idxN[len(Y_log)-1:len(Y_log)+(pred_steps)],columns=['Predicted'])
print 'FORECAST VALUES'
print print_df

FORECAST VALUES
             Predicted
2017-01-24  113.803494
2017-01-25  113.744302
2017-01-26  113.909260
2017-01-27  113.889741
2017-01-28  113.932564
2017-01-29  113.999156
2017-01-30  114.048157
2017-01-31  114.104213
2017-02-01  114.160330
2017-02-02  114.214612
2017-02-03  114.269717
2017-02-04  114.324705
2017-02-05  114.379570
2017-02-06  114.434532
2017-02-07  114.489470
2017-02-08  114.544402
2017-02-09  114.599342
2017-02-10  114.654280
2017-02-11  114.709217
2017-02-12  114.764154
2017-02-13  114.819092
2017-02-14  114.874029
2017-02-15  114.928967
2017-02-16  114.983904
2017-02-17  115.038842
2017-02-18  115.093779
2017-02-19  115.148716
2017-02-20  115.203654
2017-02-21  115.258591
2017-02-22  115.313529
...                ...
2017-03-26  117.071528
2017-03-27  117.126465
2017-03-28  117.181403
2017-03-29  117.236340
2017-03-30  117.291278
2017-03-31  117.346215
2017-04-01  117.401153
2017-04-02  117.456090
2017-04-03  117.511027
2017-04-04  117.565965
2017-04-05  117.62

## Gridsearch ARIMA order parameters

In [22]:
# Gridsearch the best (p,q,d) ordr for the ARIMA model
from sklearn.metrics import mean_squared_error
def evaluate_arima_model(X, arima_order):
    # prepare training dataset
    train_size = int(len(X) * 0.66)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit(disp=0)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    error = mean_squared_error(test, predictions)
    return error
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    mse = evaluate_arima_model(dataset, order)
                    if mse < best_score:
                        best_score, best_cfg = mse, order
                        print('ARIMA%s MSE=%.3f' % (order,mse))
                except:
                    continue
    print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))

### Find the best fit

In [23]:
# p_values = [0,1,2,3,4] # seeding from pacf vals
# d_values = range(0, 3)
# q_values = range(0, 3)
# import warnings
# warnings.filterwarnings("ignore")
# evaluate_models(Y_log, p_values, d_values, q_values)